In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [2]:
# ID => This identifies the column name in your dataset that contains a unique identifier for each sample.
ID = ["ID"]
TARGET = ["predefinedlabel"] # Label or output you're trying to predict.
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]

# A random seed ensures reproducibility.
SEED = 5412

# the number of samples to be used for training.
NUM_TRAIN_SAMPLES = 70

## Read data

- `SubjectID` = identifies each participant.
- `VideoID` = identifies each video stimulus shown to participants.
- `np.unique(data["VideoID"])` = counts how many unique videos there are.
- `len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]` = generates a unique number for each combination of subject and video.

In [6]:
data = pd.read_csv("EEG_data.csv")

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")

data = data[ID + FEATURES + TARGET] # Keep the needed column

data.head(3)

,ID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0


In [8]:
# reshape_dataset() - converts raw dataframe into
# X: 2D numpy array → each row = one trial (flattened features)
# y: 1D array of class labels for each trial

def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()): # Loops over each unique ID (i.e., each trial)
        cur_id_data = data[data[ID].to_numpy() == cur_id] # Selects all rows corresponding to the current trial
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int")) # Take the mean of the target
        features.append(cur_id_data[FEATURES].to_numpy()) # Appends the feature matrix for this trial (rows × EEG bands)

    features = pad_sequences(features)
    return np.array(features).reshape(features.shape[0], -1), np.array(target)


# EEG trials may vary in length (e.g., one trial has 5 rows, another has 7),
# but ML models expect equal-sized inputs.
def pad_sequences(arrays, pad_value=0):
    # Pads shorter trials with zeros so all have the same number of rows.
    max_length = max(arr.shape[0] for arr in arrays)
    # Finds the longest trial (in terms of number of rows/time windows)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    # Pads each array with zeros at the bottom to make it as long as max_length.
    return np.stack(padded_arrays)

In [7]:
# Train-test split

np.random.seed(SEED)

unique_ids = np.unique(np.ravel(data[ID]))
train_id = np.random.choice(unique_ids, NUM_TRAIN_SAMPLES, replace=False)
train_index = np.isin(data[ID], train_id)

train = data.iloc[train_index].copy()
test = data.iloc[~train_index].copy()

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (8870, 10)
Test shape: (3941, 10)


In [33]:
# Scaling the data

scaler = StandardScaler()
train[FEATURES] = scaler.fit_transform(train[FEATURES])
test[FEATURES] = scaler.transform(test[FEATURES])

train.head(3)

,ID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,-0.487284,-0.325816,-0.110513,-0.154594,0.106498,0.193042,0.113539,-0.179235,0.0
1,0,-0.843171,-0.588248,-0.564010,-0.537349,-0.593586,-0.470878,-0.334422,-0.362733,0.0
2,0,0.224548,0.904453,2.252233,0.516136,0.338387,1.562871,0.498640,0.384545,0.0


In [42]:
X_train, y_train = reshape_dataset(train)
X_test, y_test = reshape_dataset(test)

print(f"Reshaped Train shapes: {X_train.shape, y_train.shape}")
print(f"Reshaped Test shapes: {X_test.shape, y_test.shape}")

Reshaped Train shapes: ((70, 1152), (70,))
Reshaped Test shapes: ((30, 1152), (30,))


In [43]:
print(X_train[0, 0:8])
print(X_train[0, 8:16])
print(X_train[0, 16:24])

[-0.48728418 -0.32581625 -0.11051269 -0.15459446  0.10649789  0.19304187
  0.11353916 -0.17923454]
[-0.8431708  -0.58824825 -0.56400983 -0.53734868 -0.59358552 -0.47087767
 -0.33442203 -0.36273309]
[0.22454833 0.90445258 2.25223324 0.51613608 0.33838663 1.56287095
 0.49863985 0.38454518]


## Import Classical ML Models

Below we explore 10 classical machine learning models applied to the EEG dataset. All models use the same train/test split and reshaped feature matrix as the linear models.

In [45]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

import warnings
warnings.filterwarnings('ignore')

DEFAULT_THR = 0.5

## Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

preds = log_reg.predict(X_test)
binary_preds = (preds > DEFAULT_THR).astype("int")

print(f"ROC AUC score: {roc_auc_score(y_true=y_test, y_score=preds)}")
print(f"F1 score: {f1_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Accuracy score: {accuracy_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Precision score: {precision_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Recall score: {recall_score(y_true=y_test, y_pred=binary_preds)}")

ROC AUC score: 0.6459330143540671
F1 score: 0.6
Accuracy score: 0.6
Precision score: 0.47368421052631576
Recall score: 0.8181818181818182


## Ridge Classifier

In [54]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()
ridge.fit(X_train, y_train)

preds = ridge.predict(X_test)
binary_preds = (preds > DEFAULT_THR).astype("int")

print(f"ROC AUC score: {roc_auc_score(y_true=y_test, y_score=preds)}")
print(f"F1 score: {f1_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Accuracy score: {accuracy_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Precision score: {precision_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Recall score: {recall_score(y_true=y_test, y_pred=binary_preds)}")

ROC AUC score: 0.6722488038277513
F1 score: 0.6206896551724138
Accuracy score: 0.6333333333333333
Precision score: 0.5
Recall score: 0.8181818181818182


## SGD Classifier

In [55]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(X_train, y_train)

preds = sgd.predict(X_test)
binary_preds = (preds > DEFAULT_THR).astype("int")

print(f"ROC AUC score: {roc_auc_score(y_true=y_test, y_score=preds)}")
print(f"F1 score: {f1_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Accuracy score: {accuracy_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Precision score: {precision_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Recall score: {recall_score(y_true=y_test, y_pred=binary_preds)}")

ROC AUC score: 0.6602870813397128
F1 score: 0.5833333333333334
Accuracy score: 0.6666666666666666
Precision score: 0.5384615384615384
Recall score: 0.6363636363636364


In [ ]:
# # Prepare reshaped and scaled data
# X_train, y_train = reshape_dataset(train)
# X_test, y_test = reshape_dataset(test)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

## Support Vector Machine (SVM)

The Support Vector Machine is a supervised learning algorithm that tries to find the best decision boundary (hyperplane) that separates different classes with the maximum margin.

In [75]:
from sklearn.svm import SVC

model_svm = SVC(kernel='rbf', probability=True, random_state=SEED)
model_svm.fit(X_train, y_train)
binary_preds = model_svm.predict(X_test)

print(f"ROC AUC score: {roc_auc_score(y_true=y_test, y_score=binary_preds)}")
print(f"F1 score: {f1_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Accuracy score: {accuracy_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Precision score: {precision_score(y_true=y_test, y_pred=binary_preds)}")
print(f"Recall score: {recall_score(y_true=y_test, y_pred=binary_preds)}")

# print(classification_report(y_test, y_pred_svm))

ROC AUC score: 0.6722488038277513
F1 score: 0.6206896551724138
Accuracy score: 0.6333333333333333
Precision score: 0.5
Recall score: 0.8181818181818182


## Decision Tree

A Decision Tree splits the data into branches to make decisions based on feature values. It is easy to interpret and visualize.

In [64]:
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(random_state=SEED)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.73      0.58      0.65        19
           1       0.47      0.64      0.54        11

    accuracy                           0.60        30
   macro avg       0.60      0.61      0.59        30
weighted avg       0.64      0.60      0.61        30



## Random Forest

Random Forest is an ensemble method that uses multiple decision trees and combines their outputs to improve performance and reduce overfitting.

In [65]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=SEED)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.86      0.63      0.73        19
           1       0.56      0.82      0.67        11

    accuracy                           0.70        30
   macro avg       0.71      0.72      0.70        30
weighted avg       0.75      0.70      0.71        30



## K-Nearest Neighbors (k-NN)

K-NN is a non-parametric method that classifies a data point based on how its neighbors are classified.

In [66]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)

print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.62      0.42      0.50        19
           1       0.35      0.55      0.43        11

    accuracy                           0.47        30
   macro avg       0.48      0.48      0.46        30
weighted avg       0.52      0.47      0.47        30



## Gradient Boosting

Gradient Boosting builds an ensemble of weak learners (usually decision trees) in a stage-wise manner to minimize a loss function.

In [67]:
from sklearn.ensemble import GradientBoostingClassifier
model_gb = GradientBoostingClassifier(random_state=SEED)
model_gb.fit(X_train, y_train)
y_pred_gb = model_gb.predict(X_test)
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

           0       0.86      0.63      0.73        19
           1       0.56      0.82      0.67        11

    accuracy                           0.70        30
   macro avg       0.71      0.72      0.70        30
weighted avg       0.75      0.70      0.71        30



## Gaussian Naive Bayes

Naive Bayes uses Bayes' Theorem assuming independence between features. It's simple and works surprisingly well for many problems.

In [68]:
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)
y_pred_nb = model_nb.predict(X_test)
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       1.00      0.74      0.85        19
           1       0.69      1.00      0.81        11

    accuracy                           0.83        30
   macro avg       0.84      0.87      0.83        30
weighted avg       0.89      0.83      0.84        30



## Multi-Layer Perceptron (MLP)

MLP is a type of neural network composed of layers of nodes. It can learn complex relationships in the data.

In [69]:
from sklearn.neural_network import MLPClassifier
model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=SEED)
model_mlp.fit(X_train, y_train)
y_pred_mlp = model_mlp.predict(X_test)
print(classification_report(y_test, y_pred_mlp))

              precision    recall  f1-score   support

           0       0.75      0.63      0.69        19
           1       0.50      0.64      0.56        11

    accuracy                           0.63        30
   macro avg       0.62      0.63      0.62        30
weighted avg       0.66      0.63      0.64        30



## Make a pipeline all at once

In [72]:
# Prepare data
X_train, y_train = reshape_dataset(train)
X_test, y_test = reshape_dataset(test)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = {
    'SVM (RBF Kernel)': SVC(kernel='rbf', probability=True, random_state=SEED),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=SEED),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Gradient Boosting': GradientBoostingClassifier(random_state=SEED),
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=SEED),
    'MLP (Neural Net)': MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=SEED)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"{name} Results:")
    print(classification_report(y_test, y_pred))
    print("-" * 60)
    results.append((name, acc, f1))

SVM (RBF Kernel) Results:
              precision    recall  f1-score   support

           0       0.83      0.53      0.65        19
           1       0.50      0.82      0.62        11

    accuracy                           0.63        30
   macro avg       0.67      0.67      0.63        30
weighted avg       0.71      0.63      0.64        30

------------------------------------------------------------
Random Forest Results:
              precision    recall  f1-score   support

           0       0.86      0.63      0.73        19
           1       0.56      0.82      0.67        11

    accuracy                           0.70        30
   macro avg       0.71      0.72      0.70        30
weighted avg       0.75      0.70      0.71        30

------------------------------------------------------------
K-Nearest Neighbors Results:
              precision    recall  f1-score   support

           0       0.75      0.16      0.26        19
           1       0.38      0.91    

In [73]:
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'F1 Score'])
results_df.sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

,Model,Accuracy,F1 Score
0,Gaussian Naive Bayes,0.833333,0.836139
1,Random Forest,0.700000,0.705051
2,Gradient Boosting,0.700000,0.705051
3,MLP (Neural Net),0.700000,0.705051
4,SVM (RBF Kernel),0.633333,0.636188
5,Decision Tree,0.600000,0.607240
6,K-Nearest Neighbors,0.433333,0.363416
